In [ ]:
#  !pip install langchain faiss-cpu sentence-transformers openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "__"  # Your key from OpenRouter, Groq, etc.

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    model="shisa-ai/shisa-v2-llama3.3-70b:free",
    temperature=0,
    max_tokens=512
)


/tmp/ipython-input-2268065491.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open("ocr_text_with_tables.txt", "r", encoding="utf-8") as f:
    doc_text = f.read()

splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=100)
text_chunks = splitter.split_text(doc_text)

text_docs = [{"type": "text", "content": chunk, "source": "ocr_text"} for chunk in text_chunks]
print(f"[✓] {len(text_docs)} text chunks created.")


[✓] 94 text chunks created.


In [ ]:
import json
from hashlib import md5

def table_to_md(rows):
    md = []
    for i, row in enumerate(rows):
        line = "| " + " | ".join(str(cell) for cell in row) + " |"
        md.append(line)
        if i == 0:
            md.append("| " + " | ".join(["---"] * len(row)) + " |")
    return "\n".join(md)

table_docs = []

for fname in os.listdir("Altered_tables"):
    if fname.endswith(".json"):
        with open(f"Altered_tables/{fname}", "r", encoding="utf-8") as f:
            rows = json.load(f)
            table_md = table_to_md(rows)
            table_docs.append({
                "type": "table",
                "content": table_md,
                "source": fname
            })

print(f" {len(table_docs)} table chunks created.")


 10 table chunks created.


In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

all_docs = []

for item in text_docs + table_docs:
    all_docs.append(Document(
        page_content=item["content"],
        metadata={"source": item["source"], "type": item["type"]}
    ))

db = FAISS.from_documents(all_docs, embedder)
print("[✓] All data embedded and indexed in FAISS.")


/tmp/ipython-input-245979736.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

[✓] All data embedded and indexed in FAISS.


In [ ]:
!pip install openai==0.28

import openai


openai.api_key = "__"  # open router api key
openai.api_base = "https://openrouter.ai/api/v1"


response = openai.ChatCompletion.create(
    model="shisa-ai/shisa-v2-llama3.3-70b:free",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of Japan?"}
    ]
)

print(response['choices'][0]['message']['content'])


The capital of Japan is **Tokyo**. 

It's also the largest city in Japan, and one of the most populous metropolitan areas in the world!


In [ ]:
def ask(query, k=5):
    rel_docs = db.similarity_search(query, k=k)

    context = "\n\n---\n\n".join([doc.page_content for doc in rel_docs])
    prompt = f"""
You are a knowledgeable assistant specialized in analyzing NHPC HRA policy documents.
Answer the query by performing step-by-step logical reasoning using the provided documents.

If a direct answer is not available, use inferred logic based on the rules in the document.

If a city name is not explicitly mentioned in the documents or annexures, assume the default classification as:
- Z-Class (for cities with population below 5 lakh or if population data is missing)

Clearly explain such deductions when used.

Always follow this format:
1. **Interpretation of the question**
2. **Relevant rule extraction**
3. **Reasoning or inference**
4. **Final answer**

### Query:
{query}

### Documents:
{context}
"""
    res = llm.predict(prompt)
    return res



In [ ]:
response = ask("My grade is E6 and i reside in city classified as X what is my revised rate of house rent recovery ")
print(response)


/tmp/ipython-input-2317282374.py:28: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = llm.predict(prompt)


### Query Analysis:  
**Interpretation of the question:**  
The employee is in Grade E6 and resides in a city classified as "X." The query asks for the **revised rate of House Rent Recovery (HRR)** for the provided accommodation.  

### Relevant Rule Extraction:  
From the provided documents:  
1. **HRR Rates Table (Page 1):**  
   - For Grade **E6 & above** in **X-class** cities, the HRR is **Rs. 2750/month**.  
2. **Note on Project/Power Station Township (Page 1):**  
   - If the accommodation is in a project township, HRR is **60% of the notified rate** (rounded to the nearest 10).  
   - However, the query does not mention a project township, so this rule is not applicable.  

### Reasoning or Inference:  
- The employee is in Grade E6, which falls under the **"E6 & above"** category.  
- The city is explicitly classified as **"X"**, so no inference is needed.  
- The standard HRR rate for **E6 & above in X-class cities** is **Rs. 2750/month** (as per the table).  
- No additional 

In [ ]:
response = ask("What does the document discuss about ?")
print(response)

### **Query Interpretation**  
The query asks for a general summary of what the provided NHPC HRA policy documents discuss.  

### **Relevant Rule Extraction**  
The documents cover the following key aspects:  
1. **Definitions** (General Conditions):  
   - "Corporation" refers to NHPC Limited.  
   - "Competent Authority" is the Chairman and Managing Director.  
   - "Pay" includes Basic Pay + Stagnation Increment + NPA (for medical executives on CDA scales).  
   - "Family" includes spouse, dependent children, and wholly dependent parents.  

2. **HRA Eligibility & Certification** (Annexure-III):  
   - HRA is granted if the employee is not occupying NHPC-provided accommodation.  
   - Claims must adhere to governing orders.  

3. **Retention of Accommodation & HRA** (Undertaking in April every year):  
   - Employees must declare if they retain family at the previous posting place while drawing HRA.  

4. **HRA on Superannuation** (Section 7):  
   - The document briefly mentions H

In [ ]:
response = ask("I live in Delhi , accordingly what will be mu HRA rate per month  ?")
print(response)

### Query:  
*I live in Delhi, accordingly what will be my HRA rate per month?*  

### Step-by-Step Analysis:  

1. **Interpretation of the question**  
   - The employee lives in **Delhi** and wants to know their HRA rate per month.  
   - The query does not mention the employee’s **Basic Pay** or **IDA (Industrial Dearness Allowance) percentage**, which are required to calculate the exact HRA amount.  
   - However, the document provides HRA rates as a **percentage of Basic Pay**, and additional conditions for IDA crossing 25% and 50%.  

2. **Relevant rule extraction**  
   - **Delhi** is classified as an **X-Class city** (population > 50 Lakh).  
   - The base HRA rate for X-Class cities is **24% of Basic Pay** (Table 1).  
   - If IDA crosses **25%**, the HRA rate increases to **27%** (as per the note).  
   - If IDA crosses **50%**, the HRA rate further increases to **30%**.  
   - The minimum HRA rates are **Rs. 5,400** (Table 2).  

3. **Reasoning or inference**  
   - Since th

In [ ]:
response = ask("Who are excluded from the scope of NHPC House Rent Allowance Rules?")
print(response)

### Query:  
**Who are excluded from the scope of NHPC House Rent Allowance Rules?**  

### Step-by-Step Analysis:  

1. **Interpretation of the question**  
   - The query seeks to identify categories of employees who are **not covered** under the NHPC House Rent Allowance (HRA) Rules.  

2. **Relevant rule extraction**  
   - The exclusion criteria are explicitly mentioned in **Section 1: Title and Scope** of the NHPC HRA Rules.  
   - The excluded categories are:  
     - (a) Casual employees (including daily-rated employees)  
     - (b) Apprentices recruited under the Apprentices Act  
     - (c) Trainee Workmen & Trainee Supervisors  

3. **Reasoning or inference**  
   - The document clearly states that the HRA Rules apply to **all regular employees** of NHPC, **except** the specified categories.  
   - No additional inferences are required as the exclusions are explicitly listed.  

4. **Final answer**  
   - **The following categories are excluded from the scope of NHPC House 

In [ ]:
response = ask("Is HRA payable based on the employee’s place of residence or duty? Explain with an example.")
print(response)

### Query:  
Is HRA payable based on the employee’s place of residence or duty? Explain with an example.  

### Step-by-Step Analysis:  

1. **Interpretation of the question**  
   - The query asks whether HRA is determined by the employee's place of residence or place of duty.  
   - It also seeks an illustrative example.  

2. **Relevant rule extraction**  
   - Document (i) states: *"HRA shall be admissible with reference to place of duty and not with reference to place of residence."*  
   - Document (iii) clarifies: *"In the case of an employee who owns a house at a place of duty but resides in a rented house instead, House Rent Allowance shall be paid in respect of the rented house, if otherwise admissible."*  

3. **Reasoning or inference**  
   - The policy explicitly states that HRA is based on the **place of duty**, not residence.  
   - However, if an employee owns a house at the duty station but lives in a rented house, HRA is still payable for the rented accommodation (Doc

In [ ]:
response = ask("Jammu (UA) will be classifed in which city ? and what will be my HRA rate ")
print(response)

### Query:  
**Jammu (UA) will be classified in which city? and what will be my HRA rate?**  

### Step-by-Step Analysis:  

1. **Interpretation of the question**  
   - The query asks for the city classification of **Jammu (UA)** and the applicable **HRA rate**.  

2. **Relevant rule extraction**  
   - The document provides a table for **city classification** and corresponding **HRA rates** (Para 8).  
   - Additionally, **Annexure-I** (DoE OM No. 2/5/2017-E-II (B) dated 07.07.2017) is referenced for city classifications.  
   - The document lists **Jammu (UA)** under **Jammu & Kashmir** (Para 15).  

3. **Reasoning or inference**  
   - **Jammu (UA)** is listed in the document but not explicitly classified (X, Y, or Z).  
   - Since **Jammu (UA)** is mentioned in the list of cities (Para 15) but not in the **X or Y class** cities (which are explicitly listed in other parts of the document), we must infer its classification.  
   - The default classification for cities not explicitly

In [ ]:
response = ask("HRA rate in Itanagar? and what is the city classifed as ?")
print(response)

### Query:  
**HRA rate in Itanagar? and what is the city classified as?**  

### Step-by-Step Analysis:  

1. **Interpretation of the question**  
   - The query asks for the **HRA rate applicable in Itanagar** and its **city classification** (X, Y, or Z).  

2. **Relevant rule extraction**  
   - The HRA rates are based on city classification (X, Y, Z) as per the table:  
     | **City Class** | **HRA Rate** |  
     |--------------|-------------|  
     | X-Class      | 24% of Basic Pay |  
     | Y-Class      | 16% of Basic Pay |  
     | Z-Class      | 8% of Basic Pay |  
   - The classification is based on population (X: ≥50L, Y: 5L-50L, Z: <5L).  
   - The document does not explicitly list Itanagar in Annexure-I or any special orders.  

3. **Reasoning or inference**  
   - Since **Itanagar is not explicitly mentioned** in the provided documents or annexures, we must assume the **default classification**.  
   - The default classification for cities **not listed or with missing 

In [ ]:
response = ask("HRA rate in Meerut?")
print(response)

In [ ]:
response = ask("If my dependents reside in Faridabad and i am working in Assam , what will be my HRA rate ?")
print(response)

In [ ]:
response = ask("Can i draw HRA after retirement ?")
print(response)

In [ ]:
response = ask("Can i draw self lease after retirement ?")
print(response)

### Query:  
**Can I draw self-lease after retirement?**  

### Step-by-Step Analysis:  

1. **Interpretation of the question**  
   - The query asks whether an employee can claim House Rent Allowance (HRA) for self-owned accommodation after retirement.  

2. **Relevant rule extraction**  
   - **7.1.2** states: *"No House Rent allowance as aforesaid shall, however, be payable if such employees have been living in the accommodation owned by them or their spouse."*  
   - **iii)** states: *"In the case of an employee who owns a house at a place of duty but resides in a rented house instead, House Rent Allowance shall be paid in respect of the rented house, if otherwise admissible."*  

3. **Reasoning or inference**  
   - Rule **7.1.2** explicitly denies HRA if the employee lives in self-owned accommodation.  
   - Rule **iii)** allows HRA for rented accommodation even if the employee owns a house at the duty station, but this is for serving employees, not retirees.  
   - Post-retireme

In [ ]:
response = ask("I resided for 15 days in Y city and for 15 days in Z classified city , what will be my HRA rate ?")
print(response)

### Query:  
*I resided for 15 days in Y city and for 15 days in Z classified city, what will be my HRA rate?*  

### Step-by-Step Analysis:  

1. **Interpretation of the question**  
   - The employee spends equal time (15 days each) in a **Y-class city** and a **Z-class city**.  
   - The query asks for the applicable **HRA rate**.  

2. **Relevant rule extraction**  
   - The HRA rates are based on city classification (X, Y, Z) and are a percentage of basic pay.  
   - The default rates (as of 15.05.2018) are:  
     - **Y-class**: 16% of Basic Pay  
     - **Z-class**: 8% of Basic Pay  
   - The rules do not explicitly state how to calculate HRA when an employee resides in multiple cities.  

3. **Reasoning or inference**  
   - Since the policy does not provide a direct method for split residency, we must infer the logic.  
   - A reasonable approach is to **proportionally calculate HRA** based on the time spent in each city.  
   - Total days = 15 (Y-class) + 15 (Z-class) = 30 da

In [ ]:
response = ask("Can the family of a deceased NHPC employee continue to draw HRA? If yes, under what conditions and for how long?")
print(response)

### Query:  
Can the family of a deceased NHPC employee continue to draw HRA? If yes, under what conditions and for how long?  

### Step-by-Step Analysis:  

1. **Interpretation of the question**  
   - The query asks whether the family of a deceased NHPC employee can continue receiving House Rent Allowance (HRA) and, if so, under what conditions and for how long.  

2. **Relevant rule extraction**  
   - **Rule 9**: If an NHPC employee receiving HRA dies, the allowance may be paid to the family for **up to four months**, provided they continue residing in the same accommodation.  
   - **Conditions (Rule 9(i) to (iv))**:  
     - The HRA amount cannot exceed the last drawn by the employee or the actual rent paid, whichever is lower.  
     - Rent receipts must be produced every month (no concession for claiming without receipts).  
     - HRA is not admissible if the family resides in a house owned by the deceased or their family.  
     - "Family" includes spouse, dependent children